In [1]:
from shapely.geometry import Polygon, Point, mapping
from shapely.ops import cascaded_union
import folium
p1 = Polygon([
      (
        -72.271843,
        43.0091662
      ),
      (
        -72.2728729,
        42.9503918
      ),
      (
        -72.2230911,
        42.9486327
      ),
      (
        -72.2230911,
        43.007911
      ),
      (
        -72.271843,
        43.0091662
      )
    ])
p2 = Polygon([
      (
        -72.1870422,
        43.0048982
      ),
      (
        -72.2454071,
        42.9933479
      ),
      (
        -72.1935654,
        42.9790325
      ),
      (
        -72.2419739,
        42.9664724
      ),
      (
        -72.1746826,
        42.9586839
      ),
      (
        -72.1870422,
        43.0048982
      )
    ])
print(p1.intersects(p2))

def merge(p1, p2, intersection_with_former=True):
    if intersection_with_former:
        p2new = p2.difference(p1)
        p1new = p1
    else:
        p1new = p1.difference(p2)
        p2new = p2
    return p1new, p2new
p1new, p2new = (p1.difference(p2), p2.difference(p1))
m = folium.Map(location=[42.9933479, -72.2454071],
                   tiles='OpenStreetMap', zoom_start=12)
# m.choropleth(geo_data=p1new)
# m.choropleth(geo_data=p2new)
# m

True


In [2]:
x = p1.intersection(p2)
x1 = x
# buffer is because, polyfill returns hexgrids totally contained within the polygon. thus leaving some gaps
# between intersection and original polygons
x = x.buffer(0.001)
# m.choropleth(geo_data=x)
m

In [3]:
from h3 import h3
import geopandas as gpd
import json
polygons_x = []

for coordinates in mapping(x)["coordinates"]:
    polygon_boundary = []
    for coordinate in coordinates[0]:
        polygon_boundary.append([coordinate[0], coordinate[1]])
    geojson_x = {
        "type": "Polygon",
        "coordinates": [polygon_boundary]
    }
    polygons_x.append(geojson_x)

In [4]:
# resolution here is critical. in this case if given <=11 or 12, the hexagon is really large compared 
# to size of polygons, thus giving weird outcomes. 

# it can be mapped to area while making the function

# for example a big polgon if run on resolution 15 would take a lot of time, because the number of grids will be
# too many. 
# at the same time, very smaller resolution would lead to less fine division of area.
hexgrids_x = []
for poly in polygons_x:
    hexgrids_x.append(h3.polyfill(poly, 12))

In [5]:

p1new_buffer = []
p2new_buffer = []
p1new_buffer.append(p1new)
p2new_buffer.append(p2new)
for hexgrids in hexgrids_x:
    for cell in hexgrids:
        bound = h3.h3_to_geo_boundary(cell)
        center = Point(h3.h3_to_geo(cell))
        poly = Polygon(bound)
        d1 = center.distance(p1new)
        d2 = center.distance(p2new)
        if d1 > d2:
            p2new_buffer.append(poly)
        else:
            p1new_buffer.append(poly)
p1new_merged = cascaded_union(p1new_buffer)
p2new_merged = cascaded_union(p2new_buffer)
    


In [408]:
# m.choropleth(geo_data=x1)
# m

In [8]:
p1p2 = [p1, p2]
p1p2 = cascaded_union(p1p2)
p1p2_extended = [p1new_merged, p2new_merged]
p1p2_extended = cascaded_union(p1p2_extended)
extra_poly = p1p2_extended.difference(p1p2)
p1new = p1new_merged.difference(extra_poly)
p2new = p2new_merged.difference(extra_poly)

In [9]:
m.choropleth(geo_data=p1new)
m.choropleth(geo_data=p2new)
m